In [2]:
import mysql.connector
import json, os, re
from datetime import datetime
import pandas as pd
import re
fund_names = ["360 ONE Mutual Fund", "Aditya Birla Sun Life Mutual Fund", "Axis Mutual Fund", "Angel One Mutual Fund",
              "BajaJ Finserv Mutual Fund", "Bandhan Mutual Fund", "Bank of India Mutual Fund", 
              "Baroda BNP Paribas Mutual Fund", "Canara Robeco Mutual Fund", "DSP Mutual Fund", 
              "Edelweiss Mutual Fund", "Franklin Templeton Mutual Fund", "Groww Mutual Fund", 
              "HDFC Mutual Fund", "Helios Mutual Fund", "HSBC Mutual Fund", "ICICI Prudential Mutual Fund",
              "Invesco Mutual Fund", "ITI Mutual Fund", "JM Financial Mutual Fund", "Kotak Mahindra Mutual Fund",
              "PGIM India Mutual Fund", "LIC Mutual Fund", "Mahindra Manulife Mutual Fund", "Mirae Asset Mutual Fund", 
              "Motilal Oswal Mutual Fund", "Navi Mutual Fund", "Nippon India Mutual Fund", "NJ Mutual Fund", 
              "Old Bridge Mutual Fund", "PPFAS Mutual Fund", "Quantum Mutual Fund", "Quant Mutual Fund", 
              "Samco Mutual Fund", "SBI Mutual Fund", "Shriram Mutual fund", "Sundaram Mutual Fund", "Tata Mutual Fund", 
              "Taurus Mutual Fund", "Trust Mutual Fund", "Union Mutual Fund", "UTI Mutual Fund", "WhiteOak Mutual Fund", "Zerodha Mutual Fund"]

column_mapping = {
    "id": "id",
    "entered_time": "entered_date",
    "amc_for_month": "amc_for_month",
    "data_from": "data_from",
    "amc_name": "amc_name",
    "main_scheme_name": "main_scheme_name",
    "fund_name": "fund_name",
    "benchmark_index": "benchmark_index",
    "monthly_aaum_date": "monthly_aaum_date",
    "monthly_aaum_value": "monthly_aaum_value",
    "scheme_launch_date": "scheme_launch_date",
    "min_addl_amt": "min_addl_amt",
    "min_addl_amt_multiple": "min_addl_amt_multiple",
    "min_amt": "min_amt",
    "min_amt_multiple": "min_amt_multiple",
    "entry_load": "entry_load",
    "exit_load": "exit_load",
    "alpha": "alpha",
    "arithmetic_mean_ratio": "arithmetic_mean_ratio",
    "average_div_yld": "average_div_yield",
    "average_pb": "average_pb",
    "average_pe": "average_pe",
    "avg_maturity": "avg_maturity",
    "beta": "beta",
    "correlation_ratio": "correlation_ratio",
    "downside_deviation": "downside_deviation",
    "information_ratio": "information_ratio",
    "macaulay": "macaulay",
    "mod_duration": "mod_duration",
    "port_turnover_ratio": "port_turnover_ratio",
    "r_squared_ratio": "r_squared_ratio",
    "roe_ratio": "roe_ratio",
    "sharpe": "sharpe",
    "sortino_ratio": "sortino_ratio",
    "std_dev": "std_dev",
    "tracking_error": "tracking_error",
    "treynor_ratio": "treynor_ratio",
    "upside_deviation": "upside_deviation",
    "ytm": "ytm"
}

In [ ]:
#mydata
# conn = mysql.connector.connect(
#     host="172.22.225.155",
#     user="cog_mf",
#     password="bnYwFChjLAV2Z%9E",
#     database="cog_mf",
#     port=3306,
#     charset='utf8mb4'
# )

conn = mysql.connector.connect(
    host="localhost",   
    user="root",            
    password="1234", 
    database="cog_updated_db",  
    port=3306,               
    charset="utf8mb4"
)

# conn = mysql.connector.connect(
#     host="localhost",   
#     user="root",            
#     password="1234", 
#     database="data_db",  
#     port=3306,               
#     charset="utf8mb4"
# )

timestamp = datetime.now().strftime("%Y%m%d_%H%M")
output_path = f"march_self_{timestamp}.xlsx"
# data_from_value = "my_data"

with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    queries_dict = {}
    workbook = writer.book
    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#D7E4BC',
        'border': 1
    })
    
    for fund in fund_names:
        search_fund = re.sub(r'mutual fund', '', fund, flags=re.IGNORECASE).strip()
        query = """
        SELECT *
        FROM mf_common_fund_data
        WHERE amc_name LIKE %s
        AND amc_for_month REGEXP 'MAR25|FEB25' AND data_from LIKE "%mydata%"
        AND entered_time LIKE "%2025-05-08%";
        """
        # AND (monthly_aaum_date REGEXP "202503|March 2025" OR monthly_aaum_date IS NULL)
        # AND data_from REGEXP "mydata"
        
        # query = """
        # SELECT *
        # FROM mf_common_fund_manager_data
        # WHERE amc_name LIKE %s"""
        
        queries_dict[sheet_name] = (
            fund,
            query.strip(),
            f"LIKE: %{search_fund}%,"
        )
        
        df = pd.read_sql(query, conn, params=(f"%{search_fund}%",))

        if df.empty:
            print(f"Skipped (no data): {fund}")
            continue
        
        df.drop_duplicates(inplace=True)
        # df.drop(columns=["id","amc_id","fund_id","entered_time","amc_name","fund_name"], axis=1, inplace=True)
        
        first = "main_scheme_name"
        cols = [first]+[col for col in df.columns if col!=first]
        
        df = df[cols]
        
        sheet_name = "_".join(search_fund.lower().split())[:31]
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        worksheet = writer.sheets[sheet_name]

        for col_num, header in enumerate(df.columns.values):
            worksheet.write(0, col_num, header, header_format)
            worksheet.set_column(col_num, col_num, min(len(header) + 8, 50))

        #add query content to excel sheet
        query_rows = [
        {"Fund": fund, "Query": q, "Params": p}
        for sheet, (fund, q, p) in queries_dict.items()
        ]
        queries_df = pd.DataFrame(query_rows)
        queries_df.to_excel(writer, sheet_name="Queries", index=False)

        query_ws = writer.sheets["Queries"]
        for col_num, header in enumerate(queries_df.columns):
            query_ws.write(0, col_num, header, header_format)
            query_ws.set_column(col_num, col_num, 50)
        
print(f"Done! Exported to: {output_path}")
conn.close()

In [ ]:
#finklivedb
# query = """
#             SELECT *
#             FROM mf_json_common_details
#             WHERE (monthly_aaum_date REGEXP '2025|FEBRU' OR monthly_aaum_date IS NULL)
#               AND fund_name LIKE %s
#         """

conn = mysql.connector.connect(
    host="172.22.225.155",
    user="cog_mf",
    password="bnYwFChjLAV2Z%9E",
    database="cog_mf",
    port=3306,
    charset='utf8mb4'
)

timestamp = datetime.now().strftime("%Y%m%d_%H%M")
output_path = f"march_live_{timestamp}.xlsx"
data_from_value = "cog_mf"
aaum_date = '202503|202502|MARCH 2025|FEBRUARY 2025'

with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    queries_dict = {}
    workbook = writer.book
    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#D7E4BC',
        'border': 1
    })
    
    for fund in fund_names:
        search_fund = re.sub(r'mutual fund', '', fund, flags=re.IGNORECASE).strip()
        query = """
        SELECT 
            d.*,
            MAX(COALESCE(d.benchmark_index, bi.benchmark_index)) AS final_benchmark_index,
            MAX(bi.riskometer_benchmark) AS riskometer_benchmark,
            MAX(CASE WHEN l.type = 'Entry' THEN l.comment END) AS entry_load,
            MAX(CASE WHEN l.type = 'Exit' THEN l.comment END) AS exit_load
        FROM mf_json_common_details d
        LEFT JOIN mf_json_loads l
        ON d.MainScheme_ID = l.MainScheme_ID
        AND d.document_detail_id = l.document_detail_id
        LEFT JOIN mf_json_benchmark_indices bi
        ON d.MainScheme_ID = bi.MainScheme_ID
        AND d.document_detail_id = bi.document_detail_id
        WHERE d.fund_name LIKE %s
        AND d.monthly_aaum_date REGEXP %s
        GROUP BY d.id;
        """
        #  AND (d.monthly_aaum_date REGEXP '202503|MARCH' OR d.monthly_aaum_date IS NULL)
        
        queries_dict[sheet_name] = (
            fund,
            query.strip(),
            f"LIKE: %{search_fund}%, REGEXP: {aaum_date}"
        )
        
        
        df = pd.read_sql(query, conn, params=(f"%{search_fund}%",aaum_date))

        if df.empty:
            print(f"Skipped (no data): {fund}")
            continue
        
        #promote nonempty aaum_date
        df = df.sort_values(by='monthly_aaum_date', key=lambda col: col.isnull())
        
        df.insert(0, 'data_from', data_from_value)
        cols = df.columns.tolist()
        
        # Move 'main_scheme_name' to second column position
        if 'main_scheme_name' in cols:
            cols.remove('main_scheme_name')
            cols.insert(1, 'main_scheme_name')
            df = df[cols]

        df['benchmark_index'] = df['final_benchmark_index']
        df.drop('final_benchmark_index', axis = 1, inplace=True)
        
        sheet_name = "_".join(search_fund.lower().split())[:31]
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        worksheet = writer.sheets[sheet_name]

        for col_num, header in enumerate(df.columns.values):
            worksheet.write(0, col_num, header, header_format)
            worksheet.set_column(col_num, col_num, min(len(header) + 8, 50))

        #add query content to excel sheet
        query_rows = [
        {"Fund": fund, "Query": q, "Params": p}
        for sheet, (fund, q, p) in queries_dict.items()
        ]
        queries_df = pd.DataFrame(query_rows)
        queries_df.to_excel(writer, sheet_name="Queries", index=False)

        query_ws = writer.sheets["Queries"]
        for col_num, header in enumerate(queries_df.columns):
            query_ws.write(0, col_num, header, header_format)
            query_ws.set_column(col_num, col_num, 50)
            
print(f"Done! Exported to: {output_path}")
conn.close()

In [23]:
live_data = pd.read_excel(os.path.join(os.getcwd(),"march_live_20250508_1054.xlsx"), sheet_name=None)
self_data = pd.read_excel(os.path.join(os.getcwd(),"march_self_20250508_1109.xlsx"), sheet_name=None)


fund_names = list(live_data.keys())
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
output_path = f"march_self_live_{timestamp}.xlsx"

with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    workbook = writer.book
    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#D7E4BC',
        'border': 1
    })

    for fund in fund_names:
        df_live = live_data[fund].copy()

        df_live["data_from"] = "livedb"

        df_offline = self_data.get(fund, pd.DataFrame()).copy()
        if df_offline.empty:
            df_combined = df_live
        else:
            # df_offline["data_from"] = "mydata"
            df_offline_renamed = df_offline.rename(columns=column_mapping)

            df_offline_aligned = df_offline_renamed.reindex(columns=df_live.columns)

            # Combine live + offline
            df_combined = pd.concat([df_live, df_offline_aligned], ignore_index=True)

        df_combined.to_excel(writer, sheet_name=fund, index=False)
        worksheet = writer.sheets[fund]

        for col_num, header in enumerate(df_combined.columns):
            worksheet.write(0, col_num, header, header_format)
            worksheet.set_column(col_num, col_num, min(len(header) + 8, 50))

print("Done!")


C:\Users\Kaustubh.keny\AppData\Local\Temp\ipykernel_6640\2791599203.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_live, df_offline_aligned], ignore_index=True)
C:\Users\Kaustubh.keny\AppData\Local\Temp\ipykernel_6640\2791599203.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_live, df_offline_aligned], ignore_index=True)
C:\Users\Kaustubh.keny\AppData\Local\Temp\ipykernel_6640\2791599203.py:34: FutureWarning: The behavior of Data

Done!


In [ ]:
"""
SELECT 
    d.*,
    COALESCE(d.benchmark_index, bi.benchmark_index) AS final_benchmark_index,
    bi.riskometer_benchmark,
    MAX(CASE WHEN l.type = 'Entry' THEN l.comment END) AS entry_load,
    MAX(CASE WHEN l.type = 'Exit' THEN l.comment END) AS exit_load
FROM mf_json_common_details d
LEFT JOIN mf_json_loads l
  ON d.MainScheme_ID = l.MainScheme_ID
 AND d.document_detail_id = l.document_detail_id
LEFT JOIN mf_json_benchmark_indices bi
  ON d.MainScheme_ID = bi.MainScheme_ID
 AND d.document_detail_id = bi.document_detail_id
WHERE d.fund_name LIKE %s
  AND (d.monthly_aaum_date REGEXP '202502|FEBRUARY' OR d.monthly_aaum_date IS NULL)
GROUP BY d.id;
"""